In [45]:
from torchvision import datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn
import torch
import time
from torchvision import transforms

In [46]:
# Data to tensor
data_path = '../data-unversions/p1ch7/'
tensor_cifar10 = datasets.CIFAR10(data_path, train=True, download = True , transform=transforms.ToTensor())



Files already downloaded and verified


In [47]:
# Normalizing Image Data
imgs = torch.stack([img_t for img_t, _ in tensor_cifar10], dim=3)
mean = imgs.view(3, -1).mean(dim=1)
std = imgs.view(3, -1).std(dim=1)
normalization = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean,std)])

In [48]:
train_data = datasets.CIFAR10(root= data_path, train=True, download = True, transform=normalization)
cifar10_train = torch.utils.data.DataLoader(train_data, batch_size=64, shuffle=True)

valid_data = datasets.CIFAR10(root= data_path, train=False, download = True, transform=normalization)
cifar10_valid = torch.utils.data.DataLoader(valid_data, batch_size=64, shuffle=True)

Files already downloaded and verified
Files already downloaded and verified


Training Loop

In [49]:
def training_loop(n_epochs, optimizer, model, loss_fn, train_data, valid_data):
  for epoch in range(n_epochs+1):
    for imgs, labels in train_data:
      batch = imgs.shape[0]
      out = model(imgs.view(batch,-1))
      loss_train = loss_fn(out, labels)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()

    if epoch % 50 == 0:
      print("Epoch: %d, Training Loss: %f" % (epoch, float(loss_train)))
  trainStop = time.time()
  trainDuration = trainStop - trainStart

  print(f'Total Training Time: {trainDuration} seconds')

  total = 0
  correctClass = 0
  validStart = time.time()
  with torch.no_grad():
    for imgs, labels in valid_data:
      batch = imgs.shape[0]
      out = model(imgs.view(batch, -1))
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correctClass += int((predicted == labels).sum())

  validStop = time.time()
  validDuration = validStop - validStart

  print(f'\nValidation Accuracy: {(correctClass/total) * 100}%')
  print(f'Total Validation Time: {validDuration} seconds')
  print(f'Total Runtime: {trainDuration + validDuration} seconds')


1a

In [ ]:
seq_model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.ReLU(),
    nn.Linear(512, 10))

optimizer = optim.SGD(seq_model.parameters(), lr = 0.01)

training_loop(
    n_epochs = 300,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.CrossEntropyLoss(),
    train_data = cifar10_train,
    valid_data = cifar10_valid
)

Epoch: 0, Training Loss: 2.430470
Epoch: 50, Training Loss: 1.900788
Epoch: 100, Training Loss: 1.971840


1b

In [ ]:
seq_model = nn.Sequential(
    nn.Linear(3072, 512),
    nn.ReLU(),
    nn.Linear(512, 1024),
    nn.ReLU(),
    nn.Linear(1024, 256),
    nn.ReLU(),
    nn.Linear(256,10))

optimizer = optim.SGD(seq_model.parameters(), lr=1e-3)

training(
    epochs = 300,
    optimizer = optimizer,
    model = seq_model,
    loss_fn = nn.CrossEntropyLoss(),
    train_data = cifar10_train,
    valid_data = cifar10_valid
)